In [10]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np


class MLP(pl.LightningModule):
    def __init__(self, input_features, output_features, lr=1e-3):
        super().__init__()
        # save hyperparameters to self.hparams automatically
        self.save_hyperparameters()

        # model architecture
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_features, 1024), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(1024, 512),          nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(512, 256),           nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(256, output_features)
        )

        self.criterion = nn.MSELoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        print("t:y_hat's shape", y_hat.shape)
        print("t:y's shape", y.shape)

        loss = self.criterion(y_hat, y)

        # log to both progress bar and wandb
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        print("v:y_hat's shape", y_hat.shape)
        print("v:y's shape", y.shape)
        loss = self.criterion(y_hat, y)

        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
    
class CNN(pl.LightningModule):
    def __init__(self, input_features, output_features, lr=1e-3):
        super().__init__()
        # save hyperparameters to self.hparams automatically
        self.save_hyperparameters()

        # model architecture
        self.model = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(32 * 12 * 12, output_features)
        )

        self.criterion = nn.MSELoss()

    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = self.criterion(y_hat, y)

        # log to both progress bar and wandb
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = self.criterion(y_hat, y)

        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
    

def make_dataloaders(x_train, y_train, x_val, y_val, batch_size, input_features, output_features):
    train_ds = TensorDataset(
        torch.FloatTensor(x_train).view(-1, input_features),
        torch.FloatTensor(y_train).view(-1, output_features)
    )
    val_ds = TensorDataset(
        torch.FloatTensor(x_val).view(-1, input_features),
        torch.FloatTensor(y_val).view(-1, output_features)
    )
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(val_ds, batch_size=batch_size)
    )

In [8]:
30*25*25

18750

In [4]:
def make_dataloaders_cnn(x_train, y_train, x_val, y_val, batch_size, input_features, output_features):
    train_ds = TensorDataset(
        torch.FloatTensor(x_train).view(-1, 6, 50, 50),
        torch.FloatTensor(y_train).view(-1, output_features)
    )
    val_ds = TensorDataset(
        torch.FloatTensor(x_val).view(-1, 6, 50, 50),
        torch.FloatTensor(y_val).view(-1, output_features)
    )
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(val_ds, batch_size=batch_size)
    )

In [12]:
import random
def make_dataloaders_seq2seq(
    train_data, val_data,
    input_seq_len: int = 50,
    pred_seq_len: int = 60,
    batch_size: int = 32
):
    """
    train_data, val_data: np.ndarray of shape (N, num_agents, total_time, feat_dim)
    We will:
      - encoder-seq: first `input_seq_len` timesteps of *all* agents & features
           → shape (N, input_seq_len, num_agents * feat_dim)
      - decoder-seq: next `pred_seq_len` timesteps of agent #0, only first 2 dims
           → shape (N, pred_seq_len, 2)
    """
    def prepare(data):
        N, A, T, F = data.shape
        # encoder inputs: (N, A, input_seq_len, F)
        enc = data[:, :, :input_seq_len, :]
        # move time to dim=1: (N, input_seq_len, A, F)
        enc = enc.transpose(1, 2)
        # flatten agents+features → (N, input_seq_len, A*F)
        enc = enc.reshape(N, input_seq_len, A * F)

        # decoder targets: pick agent 0, dims 0:2, timesteps [input_seq_len:input_seq_len+pred_seq_len]
        dec = data[:, 0, input_seq_len:input_seq_len + pred_seq_len, :2]
        return enc, dec

    x_tr, y_tr = prepare(train_data)
    x_val, y_val = prepare(val_data)

    train_ds = TensorDataset(
        torch.tensor(x_tr, dtype=torch.float),
        torch.tensor(y_tr, dtype=torch.float),
    )
    val_ds = TensorDataset(
        torch.tensor(x_val, dtype=torch.float),
        torch.tensor(y_val, dtype=torch.float),
    )

    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(val_ds, batch_size=batch_size)
    )

class Seq2SeqLSTM(pl.LightningModule):
    def __init__(
        self,
        input_dim: int,        # num_agents * feat_dim  (e.g. 50*6 = 300)
        hidden_dim: int,
        output_dim: int = 2,   # target feature size per step
        pred_seq_len: int = 60,
        enc_layers: int = 1,
        dec_layers: int = 1,
        teacher_forcing: float = 0.5,
        lr: float = 1e-3
    ):
        super().__init__()
        # save all of these into self.hparams
        self.save_hyperparameters()

        self.encoder = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=enc_layers,
            batch_first=True
        )
        self.decoder = nn.LSTM(
            input_size=output_dim,
            hidden_size=hidden_dim,
            num_layers=dec_layers,
            batch_first=True
        )
        self.out_proj = nn.Linear(hidden_dim, output_dim)
        self.criterion = nn.MSELoss()

    def forward(self, src, tgt=None):
        """
        src: (B, in_seq_len, input_dim)
        tgt: (B, pred_seq_len, output_dim), only used for teacher forcing
        returns: (B, pred_seq_len, output_dim)
        """
        B = src.size(0)
        L_out = self.hparams.pred_seq_len
        device = src.device

        # run encoder
        _, (h, c) = self.encoder(src)

        # first decoder input: zeros
        dec_input = torch.zeros(B, 1, self.hparams.output_dim, device=device)
        outputs = torch.zeros(B, L_out, self.hparams.output_dim, device=device)

        for t in range(L_out):
            dec_out, (h, c) = self.decoder(dec_input, (h, c))
            pred = self.out_proj(dec_out)  # (B, 1, output_dim)
            outputs[:, t, :] = pred.squeeze(1)

            # decide teacher forcing
            if (tgt is not None) and (random.random() < self.hparams.teacher_forcing):
                dec_input = tgt[:, t, :].unsqueeze(1)
            else:
                dec_input = pred

        return outputs

    def training_step(self, batch, _):
        src, tgt = batch
        pred = self(src, tgt)
        loss = self.criterion(pred, tgt)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, _):
        src, tgt = batch
        pred = self(src, None)  # no teacher forcing in val
        loss = self.criterion(pred, tgt)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


In [ ]:
input_features = 50 * 50 * 6  # = 5000
output_features = 60 * 2
train_file = np.load('data/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)
test_file = np.load('data/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)
train_len = int(0.8 * len(train_data))
val_len = len(train_data) - train_len

x_train, y_train = train_data[:train_len, :, :50, :], train_data[:train_len, 0, 50:, :2]
x_val,   y_val   = train_data[train_len:, :, :50, :], train_data[train_len:, 0, 50:, :2]
seq_train_data = train_data[:train_len, :, :50, :]
# train_loader, val_loader = make_dataloaders_cnn(
#     x_train, y_train, x_val, y_val, 32, 
#     input_features=input_features, output_features=output_features
# )
train_loader, val_loader = make_dataloaders_seq2seq(
    seq_train_data, seq_val_data,
    input_seq_len=50,
    pred_seq_len=60,
    batch_size=32
)
for data in train_loader:
    x, y = data
    print("x's shape", x.shape)
    print("y's shape", y.shape)
    break
model = CNN(
        input_features=input_features,
        output_features=output_features,
        lr=0.001
)
trainer = pl.Trainer(
        max_epochs=1,
)

trainer.fit(model, train_loader, val_loader)

train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type       | Params | Mode 
-------------------------------------------------
0 | model     | Sequential | 558 K  | train
1 | criterion | MSELoss    | 0      | train
-------------------------------------------------
558 K     Trainable params
0         Non-trainable params
558 K     Total params
2.234     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


x's shape torch.Size([32, 6, 50, 50])
y's shape torch.Size([32, 120])
                                                                           

/Users/kimmypracha/miniconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 250/250 [00:02<00:00, 101.20it/s, v_num=3, train_loss_step=1.5e+5, val_loss=7.82e+4, train_loss_epoch=7e+5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 250/250 [00:02<00:00, 100.60it/s, v_num=3, train_loss_step=1.5e+5, val_loss=7.82e+4, train_loss_epoch=7e+5]
